In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Input_matrix_generation
from keras.utils import to_categorical

In [ ]:
df = pd.read_csv('Labeled-HLA-A-0201.csv')

In [ ]:
df_dataset = df[['sequence','label']]

In [ ]:
df_dataset.shape

In [ ]:
y_data = []
for label in df_dataset['label']:
    if label == 'P':
        y_data.append([0])
    elif label == 'N':
        y_data.append([1])
    else:
        continue
y_data = np.array(y_data)

In [ ]:
y_data = to_categorical(y_data)

In [ ]:
x_data = []
for sequence in df_dataset['sequence']:
    x_data.append(Input_matrix_generation.sequence_to_matrix(sequence))
x_data = np.array(x_data, dtype = 'float32')

In [ ]:
x_data.shape

In [ ]:
x_data = x_data.reshape((x_data.shape[0],5,12,1))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_validate,y_train,y_validate = train_test_split(x_data,y_data,test_size = 0.2)

In [ ]:
import keras
from keras import layers
from keras import models

In [ ]:
model=models.Sequential()

model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=(5, 12, 1))) 
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (1, 1), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))
sgd = keras.optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.6, nesterov=True)

In [ ]:
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, 
              epochs=50, 
              batch_size=32, 
              verbose=1, 
              validation_data=(x_validate, y_validate))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('one_channel_CNN_model.h5')